## Extracting features of wallet data

In [10]:
import pandas as pd
import numpy as np

df = pd.read_csv(r'C:\Users\ayemi\OneDrive\Documents\The_Haritz\data\cleaned_wallet_transactions.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])

features = []

for wallet, group in df.groupby('wallet'):
    total_transactions = len(group)
    inflow = group[group['direction'] == 'inflow']['amount'].sum()
    outflow = group[group['direction'] == 'outflow']['amount'].sum()
    inflow_outflow_ratio = inflow / outflow if outflow != 0 else np.inf

    frist_time = group['timestamp'].min()
    last_time = group['timestamp'].max()
    duration_days = (last_time - frist_time).total_seconds() / (24 * 3600)
    transactions_per_day = total_transactions / duration_days if duration_days > 0 else total_transactions

    features.append({
        'wallet_address': wallet,
        'total_transactions': total_transactions,
        'inflow': round(inflow, 4),
        'outflow': round(outflow, 4),
        'inflow_outflow_ratio': round(inflow_outflow_ratio, 4),
        'duration_days': round(duration_days, 2),
        'transactions_per_day': round(transactions_per_day, 2)
    })

features_df = pd.DataFrame(features)
features_df.to_csv(r'C:\Users\ayemi\OneDrive\Documents\The_Haritz\data\wallet_features.csv', index=False)
print(features_df.head())

                                 wallet_address  total_transactions  inflow  \
0  5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr                 200  0.0000   
1  7HdZJzZV1Va6H2KvyYCym6q7j5BmEdHz7ikUkdNfuGcz                  45  0.7022   
2  7uayHqA68uxnC6M4E96jgscL9X7ErtkdSUTU6qcxVkqj                   2  0.0863   
3  9Y75YHyA8GBAaMsFNqfAEChfwxyvVhgVihkK9NPRWXkS                   3  0.6018   
4  Be9RC2UC4GkV97XHe31aEULMXpex5ZbDt1cUGiwZjoEg                  24  1.9977   

   outflow  inflow_outflow_ratio  duration_days  transactions_per_day  
0   0.0000                0.0000           0.00            1329230.77  
1  15.5325                0.0452           0.01               5355.37  
2   0.0863                1.0002           0.00                464.52  
3   0.0000                   inf           5.87                  0.51  
4   0.0000                   inf           3.51                  6.83  


### Token Mint Timing vs Dump Timing

In [19]:
# mint transactions
mint_df = df[df['amount'] > 0].copy()
mint_df.rename(columns={'timestamp': 'mint_timestamp'}, inplace=True)

first_mint_per_token = mint_df.groupby('token_mint')['mint_timestamp'].min().reset_index()

# dump transactions
dump_df = df[df['amount'] < 0].copy()
dump_df.rename(columns={'timestamp': 'dump_timestamp'}, inplace=True) 

dump_joined = pd.merge(dump_df, first_mint_per_token, on='token_mint', how='inner')
dump_joined['mint_timestamp'] = (dump_joined['dump_timestamp'] - dump_joined['mint_timestamp']).dt.total_seconds() / (60)

mint_dump_summary = dump_joined.groupby('wallet').agg(
    {"token_mint":"nunique",
    "mint_timestamp":"mean"}
).reset_index()

mint_dump_summary.rename(columns={"token_mint":"minted_tokens"}, inplace=True)
mint_dump_summary.rename(columns={"mint_timestamp":"avg_mint_time"}, inplace=True)
mint_dump_summary.rename(columns={"wallet":"wallet_address"}, inplace=True)

# merge with features_df
features_df = pd.merge(features_df, mint_dump_summary, on='wallet_address', how='left')
features_df['avg_mint_time'] = features_df['avg_mint_time'].fillna(0)
features_df.fillna(0, inplace=True)

# conn = sqlite3.connect(r'C:\Users\ayemi\OneDrive\Documents\The_Haritz\data\wallet_features.db')
# features_df.to_sql('wallet_features', conn, if_exists='replace', index=False)
# conn.close()

print(features_df.head())


                                 wallet_address  total_transactions  inflow  \
0  5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr                 200  0.0000   
1  7HdZJzZV1Va6H2KvyYCym6q7j5BmEdHz7ikUkdNfuGcz                  45  0.7022   
2  7uayHqA68uxnC6M4E96jgscL9X7ErtkdSUTU6qcxVkqj                   2  0.0863   
3  9Y75YHyA8GBAaMsFNqfAEChfwxyvVhgVihkK9NPRWXkS                   3  0.6018   
4  Be9RC2UC4GkV97XHe31aEULMXpex5ZbDt1cUGiwZjoEg                  24  1.9977   

   outflow  inflow_outflow_ratio  duration_days  transactions_per_day  \
0   0.0000                0.0000           0.00            1329230.77   
1  15.5325                0.0452           0.01               5355.37   
2   0.0863                1.0002           0.00                464.52   
3   0.0000                   inf           5.87                  0.51   
4   0.0000                   inf           3.51                  6.83   

   minted_tokens  avg_mint_time  
0            0.0            0.0  
1           10.0  

### Fast Dumper Logic

In [ ]:
early_buy_mmins = 10
fast_dump_mmins = 10    

first_mint_time = df[df['amount'] > 0].groupby('token_mint')['timestamp'].min().re